In [12]:
#csv: ,로 구분하여 파일 정리
# file을 읽어와, ","를 기준으로 끊어, datatype은 float32야

import numpy as np
import tensorflow as tf
    
xy = np.loadtxt('kangnam_171116_1800.csv', dtype = 'float')
print (xy)
#x_data = xy[:,1:3]
#y_data = xy[:, 8]

# Make sure the shape and data are OK
# print(x_data.shape, x_data, len(x_data))
# print(y_data.shape, y_data)

UnicodeEncodeError: 'latin-1' codec can't encode character '\ufeff' in position 0: ordinal not in range(256)

In [82]:
import xlrd
import os
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import math
from scipy import stats
from scipy.stats import multivariate_normal

class RohdeData:
    def __init__(self, band_info, timestamp, NumRNTI, RBUsage, AvgUETP, AvgCellTP, RB_Trans, CellTP_Trans):
        self.band_info = band_info
        self.timestamp = timestamp
        self.NumRNTI = NumRNTI
        self.RBUsage = RBUsage
        self.AvgUETP = AvgUETP
        self.AvgCellTP = AvgCellTP
        self.RB_Trans = RB_Trans
        self.CellTP_Trans = CellTP_Trans

class TransParameter:
    def __init__(self, band_info, NumRNTI, meanRB, meanCellTP, var_RB, var_RBTP, var_TP):
        self.band_info = band_info
        self.NumRNTI = NumRNTI
        self.meanRB = meanTransRB
        self.meanCellTP = meanTransTP
        self.var_RB = var_RB
        self.var_RBTP = var_RBTP
        self.var_TP = var_TP
        
# fileread, preprocess, data transformation
def fileread(file):
    workbook = xlrd.open_workbook(os.path.join('/Users/sjlee/Desktop/Graduate/intern_etri/LackofQoS/', file), encoding_override="utf-8")
    sheetNumber = workbook.nsheets-2   # total data 있는 sheet 미포함
    Rohdedata_list = []

    for index in range(sheetNumber):
        worksheet = workbook.sheet_by_index(index)
        worksheetName = worksheet.name
        band_info = worksheetName  # sheetName: company_band
        sample_number = int(worksheet.cell_value(0,3))
        start_col_index = 0
        start_row_index = 2
    #     Data_dic = {}
        for row_index in range(0, sample_number-2):
            cur_row_index = start_row_index + row_index
            cur_col_index = start_col_index
            timestamp = worksheet.cell_value(cur_row_index, cur_col_index)
            cur_col_index += 1
            NumRNTI = worksheet.cell_value(cur_row_index, cur_col_index)
            cur_col_index += 1
            RBUsage = worksheet.cell_value(cur_row_index, cur_col_index)
            cur_col_index += 1
            AvgUETP = worksheet.cell_value(cur_row_index, cur_col_index)
            cur_col_index += 1
            AvgCellTP = worksheet.cell_value(cur_row_index, cur_col_index)
            RB_Trans = math.log(RBUsage/(100-RBUsage))
            CellTP_Trans = math.log(AvgCellTP)
            Rohdedata = RohdeData(band_info, timestamp, NumRNTI, RBUsage, AvgUETP, AvgCellTP, RB_Trans, CellTP_Trans)
            Rohdedata_list.append(Rohdedata)
    return Rohdedata_list
    
Total_list = fileread('postseminar.xlsx')

def TransData(original_data):
    RNTI_list = []
    for i in range(len(Rohdedata_list)):
        RNTI_list.append(Total_list[i].NumRNTI)
    RNTI_list = list(set(RNTI_list))  #####RNTI 개수 어디까지 cutting! 스스로 해결!

    alpha = 1e-3
    selected_RNTI = []
    TransRB_mean = []
    TransTP_mean = []
    TransRB_var = []
    TransTP_var = []
    TransRBTP_var = []
    #parameters_list = []
    for i in range(len(RNTI_list)):
        RNTI_fixed = []
        RB_fixed = []
        CellTP_fixed = []
        for j in range(len(Total_list)):
            if Total_list[j].NumRNTI == RNTI_list[i]:
                RNTI_fixed.append(Total_list[j])        
                RB_fixed.append(Total_list[j].RB_Trans)
                CellTP_fixed.append(Total_list[j].CellTP_Trans)
        if len(RB_fixed) >= 8 and len(CellTP_fixed) >= 8:
            p_value_RB = stats.normaltest(RB_fixed).pvalue
            p_value_CellTP = stats.normaltest(CellTP_fixed).pvalue
            #p_value_joint = 
            ####python 에 multivariative test 없는지!!!!

            if p_value_RB > alpha and p_value_CellTP > alpha:
                selected_RNTI.append(RNTI_list[i])
                TransRB_mean.append(np.mean(RB_fixed))
                TransTP_mean.append(np.mean(CellTP_fixed))
                TransRB_var.append(np.cov(RB_fixed, CellTP_fixed)[0,0])
                TransTP_var.append(np.cov(RB_fixed, CellTP_fixed)[1,1])
                TransRBTP_var.append(np.cov(RB_fixed, CellTP_fixed)[0,1])
#                 meanTransRB = np.mean(RB_fixed)
#                 meanTransTP = np.mean(CellTP_fixed)
#                 var_RB = np.cov(RB_fixed, CellTP_fixed)[0,0]
#                 var_RBTP = np.cov(RB_fixed, CellTP_fixed)[0,1]
#                 var_TP = np.cov(RB_fixed, CellTP_fixed)[1,1]
#                 print(meanTransRB)
#                 parameters = TransParameter(band_info, RNTI_list[i], meanTransRB, meanTransTP, var_RB, var_RBTP, var_TP)
#                 parameters_list.append(parameters)
    return selected_RNTI, TransRB_mean, TransTP_mean, TransRB_var, TransTP_var, TransRBTP_var

selected_RNTI, TransRB_mean, TransTP_mean, TransRB_var, TransTP_var, TransRBTP_Var = TransData(Total_list) 
# selected_RNTI, parameters_list = TransData(Total_list)            
print(selected_RNTI)          
print(TransRB_mean)
# TransRB_mean = []
#TransRB_mean = [parameters_list[i].meanRB for i in range(len(parameters_list))]
#print(TransRB_mean)

[2.0, 10.0, 11.0, 16.0, 17.0, 18.0, 19.0, 20.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 29.0, 30.0, 31.0]
[-2.1508089570896436, 0.54857714382342126, 0.35804574580068182, -0.28826058849614661, -0.386213704989672, 0.11882332630375195, -0.1609996677417532, 0.083151685008133927, 0.17598964220400989, 0.12805787328961488, 0.77864191993427556, 0.5893191753744319, 0.66678426976510319, 0.84923411518219405, 1.1704377898147582, 1.2748226259224789, 1.3446021955986318]


/Users/sjlee/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1390: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))
/Users/sjlee/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1390: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))
/Users/sjlee/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1390: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/sjlee/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1390: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  "anyway, n=%i" % int(n))
/Users/sjlee/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1390: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  "anyway, n=%i" % int(n))


In [88]:
x_data = np.array(selected_RNTI, dtype = np.float32)
y_data = []
for i in range(len(selected_RNTI)):
    array_item = [TransRB_mean[i], TransTP_mean[i], TransRB_var[i], TransTP_var[i], TransRBTP_Var[i]]
    y_data.append(array_item)
y_data = np.array(y_data, dtype = np.float32)    




[[-2.15080896  6.39312989  4.98226206  3.39162143  3.56514272]
 [ 0.54857714  8.51364447  1.20655285  0.47170533  0.41006809]
 [ 0.35804575  8.54232973  1.20061918  0.39228426  0.37348489]
 [-0.28826059  8.75110989  1.42271168  0.78317774  0.81688656]
 [-0.3862137   8.73073897  1.84576253  0.60987235  0.6601968 ]
 [ 0.11882333  8.67180904  1.94029197  0.38976805  0.478883  ]
 [-0.16099967  8.69547938  1.54004143  0.66085296  0.37622932]
 [ 0.08315169  8.73893572  2.01765865  0.43100166  0.29802246]
 [ 0.17598964  8.49998267  2.51141132  0.65638468  0.68846582]
 [ 0.12805787  8.40546844  2.4556637   0.59334334  0.56882052]
 [ 0.77864192  8.7321899   2.77630136  0.31540704  0.3533298 ]
 [ 0.58931918  8.53752991  2.33116872  0.36732522  0.22251162]
 [ 0.66678427  8.42928553  1.78509222  0.51749649  0.51807889]
 [ 0.84923412  8.75215419  1.3080335   0.48737192  0.10045608]
 [ 1.17043779  8.41209533  1.1456361   0.25495319  0.30070661]
 [ 1.27482263  8.96495863  0.91375721  0.37618561 -0.17

In [ ]:
# normaltest    
# RNTI 개수를 조건부로, RB_Trans와 가 정규분포를 따르는지
# RNTI 개수를 set으로 > RNTI_list
# RNTI 개수가 i 인 element 모아서 list만들어야
# 만든 list로 normality test
# pass > class 저장: bandinfo, NumRNTI, Trans_RB, CellTP_Trans > 인스턴스 리스트로
# 리스트에 있는 item: jointnormal test

#print(Total_list[0].RB_Trans)        
        
#     for col_index in range(0, 5):
#         start_row_index = 2
#         cur_col_index = start_col_index + col_index
#         col_name = worksheet.cell_value(1, cur_col_index)
#         Data_dic[col_name] = []
        
#         for row_index in range(0, sample_number-2):
#             cur_row_index = start_row_index+row_index
#             temp_value = worksheet.cell_value(cur_row_index, cur_col_index)
#             Data_dic[col_name].append(temp_value)
#         if col_name == "RBUsage":
#             RB_temp = Data_dic["RBUsage"]
#             RB_temp[:] = [math.log(x/(100-x)) for x in RB_temp]
#             Data_dic["RB_Trans"] = RB_temp
#         if col_name == "AvgCellTP":
#             Data_dic["TP_Trans"] = [math.log(x) for x in Data_dic["AvgCellTP"]]
#         Rohdedata = RohdeData(band_info, Data_dic)
#     Rohdedata_list.append(Rohdedata)


# for i in range(len(Rohdedata_list)):
#     Rohdedata_list[i].Data_dic[""]

##def normalitytest(list?):

# x = np.concatenate((Rohdedata_list[i].Data_dic['NumRNTI'],Rohdedata_list[i].Data_dic['RB_Trans']))
# k2, p = stats.normaltest(x)
# alpha = 1e-3

# if p < alpha:  # null hypothesis: x comes from a normal distribution
# ...     print("The null hypothesis can be rejected")
# ... else:
# ...     print("The null hypothesis cannot be rejected")
# The null hypothesis can be rejected

[  2.  10.  11.  16.  17.  18.  19.  20.  22.  23.  24.  25.  26.  27.  29.
  30.  31.]


In [5]:
# placeholders for a tensor that will be always fed.
# -> Session run 할 때 값을 넣어주고, 그 전까지 자리만들기
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')   # [in, out]
b = tf.Variable(tf.random_normal([1]), name='bias')   # [out]

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# gvs = optimizer.compute_gradients(cost)
# apply_gradients = optimizer.apply_gradients(gvs)

# Minimize: tf의 variable을 조정하면서 cost를 minimize (Variable로 설정한 것만 조정)
# cost function을 굳이 미분하지 않아도 gradient구해서 최소화 (optimizer을 gradient descent로 assign했으므로)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)   # train은 node의 개념

NameError: name 'tf' is not defined

In [ ]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph. (Variable 로 assign한 것만)
sess.run(tf.global_variables_initializer())

for step in range(2001):
    ## cost_val에 cost 저장, hy_val 에 hypothesis 저장, train은 의미 없음 (-)
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    
    ##구체적으로 run에 들어가야 하는것??
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

# Ask my score
print("Your score will be ", sess.run(
    hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ", sess.run(hypothesis,
                                        feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))


In [ ]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
# 한번 펌프할 때마다 10개씩 가져오기
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    #펌프질을 해서 데이터를 가져온다
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

In [58]:
a = np.array([[1,2],[3,4]])
print(a[1,1])

4
